### Importing libraries

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC

In [40]:
df = df = pd.read_csv('nfr.csv')
df = df.dropna(axis=1, how='any')
df.head()

,Type,Requirement
0,PE,The system shall refresh the display every 60 ...
1,LF,The application shall match the color of the s...
2,US,If projected the data must be readable. On ...
3,A,The product shall be available during normal ...
4,US,If projected the data must be understandable...


In [42]:
# Divide the data into features (X) and labels (y)
X = df.drop('Type', axis=1)
y = df['Type']

In [43]:
X = X.apply(LabelEncoder().fit_transform)

In [44]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [46]:
# Train a support vector machine classification model
model = SVC()
model.fit(X_train, y_train)

SVC()

In [47]:
# Evaluate the model on the test set
accuracy = model.score(X_test, y_test)
print('Model accuracy:', accuracy)

Model accuracy: 0.375


In [53]:
# Create a list of new software requirements
new_requirements = [
    {'name': 'Login page'},
    {'name': 'Search feature'},
    {'name': 'Performance optimization'},
]

In [54]:
# Convert the new requirements to a DataFrame
new_requirements_df = pd.DataFrame(new_requirements)

In [55]:
new_requirements_df = new_requirements_df.apply(LabelEncoder().fit_transform)

In [56]:
# Standardize the data
new_requirements_df = scaler.transform(new_requirements_df)

C:\Users\Vaibhav\Desktop\CODE\MachineLearning\ML-practice\env\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- name
Feature names seen at fit time, yet now missing:
- Requirement

  warnings.warn(message, FutureWarning)


In [57]:
# Use the model to predict the labels of the new requirements
predictions = model.predict(new_requirements_df)

In [58]:
print(predictions)

['F' 'F' 'F']
